## Amazon Web Scraping using Python
### This Scraping is done on Wednesday, November 16, 2022

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find('span', attrs = {'class':'a-size-medium a-color-price'}).string.strip()

    except AttributeError:

        #try:
            # If there is some deal price
            #price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        #except:
        price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available

In [3]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'<insert your user agent>', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=laptop+i7&ref=nb_sb_noss"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a",attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [4]:
amazon_df

,title,price,rating,reviews,availability
0,"HP Business Laptop, 15.6"" FHD IPS Touchscreen ...",$799.00,4.3 out of 5 stars,2 ratings,Not Available
1,"Lenovo Ideapad 3 14"" FHD Slim Laptop, Intel Co...",$699.00,Previous page of related Sponsored Products,,Not Available
2,"HP Pavilion 15 Laptop, 11th Gen Intel Core i7-...",$784.00,4.4 out of 5 stars,680 ratings,Not Available
3,ASUS Vivobook 15.6 Full HD Intel Gen 10 Core i...,$638.00,4.6 out of 5 stars,97 ratings,Not Available
4,"2021 Newest Dell Inspiron 15 3000 3501 Laptop,...",$560.28,4.2 out of 5 stars,204 ratings,Not Available
5,"Microsoft Surface Pro 7 – 12.3"" Touch-Screen -...",$869.99,4.5 out of 5 stars,"1,192 ratings",Not Available
6,Acer Nitro 5 AN517-54-77KG Gaming Laptop | Int...,"$1,199.00",4.5 out of 5 stars,565 ratings,Only 1 left in stock - order soon
7,"Acer Aspire 5 A515-56-73AP, 15.6"" Full HD IPS ...",$999.99,4.3 out of 5 stars,188 ratings,Not Available
8,"HP Pavilion 15.6"" FHD Laptop (2022 Latest Mode...",$649.00,4.1 out of 5 stars,294 ratings,Not Available
9,"Acer Aspire Vero AV15-51-7617 Green PC | 15.6""...",$693.50,4.6 out of 5 stars,170 ratings,In Stock.
